In [1]:
# Import modules
import unidecode as ud
import lxml.etree as ET
import pprint as pp
import json
import re

# Import classes
from datetime import datetime
from pymongo import MongoClient

# Global variables
file_init ="tournefeuille.xml"
file_tagok = "tournefeuille_tagok.xml"
file_json = "tournefeuille.json"

result_dict = {}
result_list = []

########################################################################################
# Generic function to parse XML files
# This function will be called for read-only XML parsing using a custom defined function
# This function is an adaptation of Liza Daly's fast_iter function
########################################################################################



def parse(context, func, *args, **kwargs):

    global result_dict
    global result_list
    
    result_dict = {}
    result_list = []
    """
    http://lxml.de/parsing.html#modifying-the-tree
    Based on Liza Daly's fast_iter
    http://www.ibm.com/developerworks/xml/library/x-hiperfparse/
    See also http://effbot.org/zone/element-iterparse.htm
    """
    begin = datetime.now()
    for event, elem in context:
        func(elem, event, *args, **kwargs)
        # It's safe to call clear() here because no descendants will be
        # accessed
        elem.clear()
        # Also eliminate now-empty references from the root node to elem
        for ancestor in elem.xpath('ancestor-or-self::*'):
            while ancestor.getprevious() is not None:
                del ancestor.getparent()[0]
    del context
    
    # Print elapse time
    end = datetime.now()
    print "Start : " + str(begin) + " / Finish : " + str(end) + " / Duration : " + str(end - begin)
    print
    
    # print result dictionnary if not empty  
    if len(result_dict) > 0:
        pp.pprint(result_dict)
    print
    

############################################################
# Generic function used in all various computation performed
#   count   : increase the key counter in a dictionnary
#   isfloat : test is a value is a float
############################################################

def count(keyval, dictionnary):
    if keyval in dictionnary:
        dictionnary[keyval] = dictionnary[keyval] +1
    else:
        dictionnary[keyval] = 1

def isfloat(value):
    try:
        fvalue = float(value)
        return True
    except ValueError:
        return False

print "Init done"

Init done


In [2]:
## structure 1 ## 

# Objective is to get a global view of the XML file content

currentpath = []

def getstructure(elem, event):
    global result_dict
    global currentpath
    
    if event == "start":
        currentpath.append(elem.tag)
        count(' / '.join(currentpath), result_dict)
    else:
        # event = end
        last = currentpath.pop()

    
print "File content - global view"
# look to both start and end event to manage the path stack properly
context = ET.iterparse(file_init, events=('start','end'))
parse(context,getstructure)


File content - global view
Start : 2016-12-08 10:34:41.126000 / Finish : 2016-12-08 10:35:35.081000 / Duration : 0:00:53.955000

{'osm': 1,
 'osm / bounds': 1,
 'osm / meta': 1,
 'osm / node': 447694,
 'osm / node / tag': 33717,
 'osm / note': 1,
 'osm / relation': 465,
 'osm / relation / member': 17453,
 'osm / relation / tag': 2307,
 'osm / way': 73067,
 'osm / way / nd': 618158,
 'osm / way / tag': 184820}



In [3]:
## structure 2 ## 

# Objective is to identify the number of empty and non empty main level 1 tags (node, relation, way)

def countelt(elem, event):
    global result_dict
    
    if len(elem) == 0:
        count('Empty - ' + elem.tag,result_dict)
    else:
        count('Non empty - ' + elem.tag,result_dict)

print "Empty / Non emtpy tags"
# use the end event to be sure children are populated
context = ET.iterparse(file_init, events=('end',),tag=('node','relation','way'))
parse(context,countelt)


Empty / Non emtpy tags
Start : 2016-12-08 10:35:35.109000 / Finish : 2016-12-08 10:35:49.753000 / Duration : 0:00:14.644000

{'Empty - node': 434974,
 'Non empty - node': 12720,
 'Non empty - relation': 465,
 'Non empty - way': 73067}



In [4]:
## structure 3 ## 

# Check consistency between the DTD and the real data file content

# The checkvisible global variable is used to activate (or not) the visibility attribute check
checkvisible = True

def checknode(elem, event):
    global result_dict    
    global checkvisible
    
    ok = False
    if elem.tag == "node":
        # check lon and lat belong to the elem and are both float
        if 'lon' in elem.attrib and 'lat' in elem.attrib:
            ok = (isfloat(elem.attrib['lon']) and isfloat(elem.attrib['lat']))
    
    elif elem.tag == "way":    
        
        if checkvisible:
            # we check that the visible tag belong to the element
            if 'visible' in elem.attrib:
                ok = (elem.attrib['visible'] in ['true','false'])
        else:
            ok = True
            
        # we check the element contains at least 2 nd sub elements
        nb = 0
        for subelem in elem:
            if subelem.tag == "nd":
                nb = nb + 1                
        
        # We need at least 2 nd tags to be a valid way
        ok = ok and (nb >= 2)

    else:
        #relation
        if checkvisible:
            # we check that the visible tag belong to the element
            if 'visible' in elem.attrib:            
                ok = (elem.attrib['visible'] in ['true','false'])
        else:
            ok = True
        
        # we check the element contains at least 1 member sub elements
        nb = 0            
        for subelem in elem:
            if subelem.tag == "member":
                nb = nb +1                
        
        # We need at least on member to be valid        
        ok = ok and (nb >= 1)

    key = elem.tag + " - tag " + ("Ok" if ok else "Ko")
    count(key,result_dict)
    
print "Consitency with DTD and associated business rules"    

checkvisible = True
print "Including visibility attribute check"
# use the end event to be sure children are populated
context = ET.iterparse(file_init, events=('end',),tag=('node','way','relation'))
parse(context,checknode)

checkvisible = False
print "Not including visibility attribute check"
# use the end event to be sure children are populated
context = ET.iterparse(file_init, events=('end',),tag=('node','way','relation'))
parse(context,checknode)


Consitency with DTD and associated business rules
Including visibility attribute check
Start : 2016-12-08 10:35:49.844000 / Finish : 2016-12-08 10:36:06.493000 / Duration : 0:00:16.649000

{'node - tag Ok': 447694, 'relation - tag Ko': 465, 'way - tag Ko': 73067}

Not including visibility attribute check
Start : 2016-12-08 10:36:06.495000 / Finish : 2016-12-08 10:36:21.315000 / Duration : 0:00:14.820000

{'node - tag Ok': 447694, 'relation - tag Ok': 465, 'way - tag Ok': 73067}



In [5]:
## content 1 ## 

# Check the tags content for the different nodes

def counttag(elem, event):
    global result
    
    for subelem in elem:
        if subelem.tag == "tag":
            count(subelem.attrib['k'],result_dict)

def checktags(xmlfile):
    # use the end event to be sure children are populated
    print "Way tags"
    context = ET.iterparse(xmlfile, events=('end',),tag=('way'))
    parse(context,counttag)

    print "Node tags"
    context = ET.iterparse(xmlfile, events=('end',),tag=('node'))
    parse(context,counttag)

    print "Relation tags"
    context = ET.iterparse(xmlfile, events=('end',),tag=('relation'))
    parse(context,counttag)

print "Check tag content"
checktags(file_init)
    

Check tag content
Way tags
Start : 2016-12-08 10:36:21.371000 / Finish : 2016-12-08 10:36:27.432000 / Duration : 0:00:06.061000

{'CLC:code': 2,
 'CLC:id': 2,
 'CLC:year': 2,
 'FIXME': 3,
 'Levels': 10,
 'access': 3090,
 'addr:city': 51,
 'addr:housename': 23,
 'addr:housenumber': 355,
 'addr:postcode': 115,
 'addr:street': 299,
 'admin_level': 27,
 'advertising': 3,
 'aerodrome': 1,
 'aeroway': 20,
 'agricultural': 1,
 'alt_name': 12,
 'amenity': 452,
 'area': 40,
 'atm': 2,
 'barrier': 185,
 'basin': 1,
 'bench': 1,
 'bicycle': 735,
 'bicycle_parking': 1,
 'bike_safety': 53,
 'board_type': 2,
 'boat': 15,
 'boundary': 46,
 'branch': 1,
 'brand': 3,
 'bridge': 185,
 'building': 60705,
 'building:colour': 2,
 'building:levels': 36,
 'building:material': 2,
 'building:min_level': 1,
 'bus': 8,
 'busway:right': 1,
 'bylaw': 1,
 'capacity': 53,
 'capacity:disabled': 31,
 'change_machine': 1,
 'comment': 3,
 'community_centre:for': 1,
 'construction': 3,
 'contact:fax': 1,
 'contact:phone'

In [6]:
##contents 2 ## 

# Delete the tags that do not represent a Map Feature
# The list of tags representing a Map feature has be extracted mannually from : http://wiki.openstreetmap.org/wiki/Map_Features
# We will modify the file content and therefore can not use the iterator based parser we defined.


def deletetag():

    allowedtags = ['abutters','addr:city','addr:conscriptionnumber','addr:country','addr:flats','addr:full','addr:housename','addr:housenumber','addr:inclusion','addr:interpolation','addr:place','addr:postcode','addr:street','admin_level','aerialway','aeroway','amenity','attribution','barrier','bicycle_road','border_type','boundary','bridge','building','building:fireproof','building:levels','busway','comment','craft','cutting','cycleway','description','driving_side','electrified','email','embankment','emergency','entrance','fax','fire_boundary','fixme','ford','frequency','geological','height','highway','historic','ice_road','image','incline','intermittent','is_in','junction','landuse','lanes','leisure','lit','lock','man_made','max_level','military','min_level','mooring','motorroad','mountain_pass','mtb:description','mtb:scale','mtb:scale:imba','mtb:scale:uphill','name','natural','non_existent_levels','note','office','overtaking','parking:condition','parking:lane','passing_places','phone','place','population','postal_code','power','public_transport','railway','railway:track_ref','reference_point','route','sac_scale','seasonal','service','shop','source','source:name','source:ref','source_ref','sport','surface','tactile_paving','todo','tourism','tracks','tracktype','traffic_calming','trail_visibility','tunnel','url','usage','voltage','waterway','website','wikipedia','winter_road']

    tree = ET.parse(file_init)
    root = tree.getroot()
    for tag in root.iter('tag'):
        if not (tag.attrib['k'] in allowedtags):
            tag.getparent().remove(tag)
    
    ET.ElementTree(root).write(file_tagok, xml_declaration=True,encoding="UTF-8")
            
print "Remove non Map Features tags"
deletetag()
print "New file created"
            

Remove non Map Features tags
New file created


In [7]:
## content 3 ##

# Now check the new generated file tags

checktags(file_tagok)


Way tags
Start : 2016-12-08 10:36:50.908000 / Finish : 2016-12-08 10:36:57.695000 / Duration : 0:00:06.787000

{'addr:city': 51,
 'addr:housename': 23,
 'addr:housenumber': 355,
 'addr:postcode': 115,
 'addr:street': 299,
 'admin_level': 27,
 'aeroway': 20,
 'amenity': 452,
 'barrier': 185,
 'boundary': 46,
 'bridge': 185,
 'building': 60705,
 'building:levels': 36,
 'comment': 3,
 'cutting': 5,
 'cycleway': 160,
 'description': 3,
 'electrified': 58,
 'email': 2,
 'embankment': 1,
 'emergency': 8,
 'fixme': 18,
 'height': 2,
 'highway': 7621,
 'historic': 20,
 'incline': 13,
 'intermittent': 5,
 'junction': 387,
 'landuse': 266,
 'lanes': 118,
 'leisure': 3220,
 'lit': 245,
 'man_made': 21,
 'military': 1,
 'name': 3650,
 'natural': 172,
 'note': 47,
 'office': 9,
 'phone': 13,
 'power': 32,
 'public_transport': 12,
 'railway': 76,
 'service': 1134,
 'shop': 22,
 'source': 66334,
 'sport': 154,
 'surface': 604,
 'tourism': 9,
 'tracktype': 39,
 'traffic_calming': 17,
 'trail_visibilit

In [8]:
## content 4 ## 

# We check the now the tags contents
# These chekcs are defined through function because they will be reused after

def audittag(elem, event):
    global result
    
    count(elem.attrib['k'] + " / " + elem.attrib['v'],result_dict)
    
def checktagcontent():
    # use the end event to be sure children are populated
    context = ET.iterparse(file_tagok, events=('end',),tag=('tag'))
    parse(context,audittag)

print "Check tag content"
checktagcontent()


Check tag content
Start : 2016-12-08 10:37:15.629000 / Finish : 2016-12-08 10:37:23.873000 / Duration : 0:00:08.244000

{'addr:city / COLOMIERS': 2,
 'addr:city / Colomiers': 26,
 'addr:city / Cugnaux': 390,
 'addr:city / La Salvetat Saint Gilles': 1,
 'addr:city / Pibrac': 1,
 'addr:city / Plaisance du touch': 1,
 'addr:city / Plaisance-du-Touch': 2,
 'addr:city / Toulouse': 1467,
 'addr:city / Tournefeuille': 53,
 'addr:city / plaisance du touch': 1,
 'addr:country / FR': 2281,
 'addr:housename / 30': 1,
 'addr:housename / Airbus Operations SAS': 1,
 'addr:housename / Batiment A': 1,
 'addr:housename / Batiment B': 1,
 u"addr:housename / Bocage de l'Armuri\xe9 B\xe2timent A": 1,
 u"addr:housename / Bocage de l'Armuri\xe9 B\xe2timent B": 1,
 u'addr:housename / B\xe2timent Kappa': 1,
 'addr:housename / D22': 1,
 'addr:housename / D26': 1,
 'addr:housename / Europarc B1': 1,
 'addr:housename / Europarc B10': 1,
 'addr:housename / Europarc B2': 1,
 'addr:housename / Europarc B3': 1,
 'ad

In [9]:
## contents 5 ## 

# Transform unicode string to ascci strings

def deleteunicode():

    tree = ET.parse(file_tagok)
    root = tree.getroot()
    for tag in root.iter('tag'):
        if isinstance(tag.attrib['v'], unicode):
            tag.attrib['v'] = ud.unidecode(tag.attrib['v'])
    
    ET.ElementTree(root).write(file_tagok, xml_declaration=True,encoding="UTF-8")
            
print "Remove unicode string"
deleteunicode()
print "New file created"

Remove unicode string
New file created


In [10]:
## contents 6 ## 

# Recheck content
checktagcontent()


Start : 2016-12-08 10:37:36.774000 / Finish : 2016-12-08 10:37:46.121000 / Duration : 0:00:09.347000

{'addr:city / COLOMIERS': 2,
 'addr:city / Colomiers': 26,
 'addr:city / Cugnaux': 390,
 'addr:city / La Salvetat Saint Gilles': 1,
 'addr:city / Pibrac': 1,
 'addr:city / Plaisance du touch': 1,
 'addr:city / Plaisance-du-Touch': 2,
 'addr:city / Toulouse': 1467,
 'addr:city / Tournefeuille': 53,
 'addr:city / plaisance du touch': 1,
 'addr:country / FR': 2281,
 'addr:housename / 30': 1,
 'addr:housename / Airbus Operations SAS': 1,
 'addr:housename / Batiment A': 1,
 'addr:housename / Batiment B': 1,
 'addr:housename / Batiment Kappa': 1,
 "addr:housename / Bocage de l'Armurie Batiment A": 1,
 "addr:housename / Bocage de l'Armurie Batiment B": 1,
 'addr:housename / D22': 1,
 'addr:housename / D26': 1,
 'addr:housename / Europarc B1': 1,
 'addr:housename / Europarc B10': 1,
 'addr:housename / Europarc B2': 1,
 'addr:housename / Europarc B3': 1,
 'addr:housename / Europarc B4': 1,
 'ad

In [11]:
## contents 7 ## 

## Apply transformation ##

# Transformation to be applied are stored as regular expression within a dictionnary of dictionnary
# The main dictionnary key is the tag on which transformation has to be applied
# The second dictionnary contains the searched pattern (key) and the replacement string (value)

def applymod():

    replace = {
        'addr:city' : { 'COLOMIERS' : 'Colomiers', 'Plaisance-du-Touch' : 'Plaisance du Touch', 'Plaisance du touch' : 'Plaisance du Touch', 'plaisance du touch' : 'Plaisance du Touch' }
        , 'addr:street' : { '\\brue\\b' : 'Rue', '\\ballee\\b' : 'Allee', '\\bavenue\\b' : 'Avenue' , '\\bimpasse\\b' : 'Impasse', '\\bplace\\b' : 'Place' , '\\bAv\\.\\b' : 'Av', '\btouch\b' : 'Touch'}
        , 'addr:postcode' : { '31170 Tournefeuille' : '31170', '31000;31100;31200;31300;31400;31500' : '31000'}
        }

    tree = ET.parse(file_tagok)
    root = tree.getroot()
    for tag in root.iter('tag'):
        if tag.attrib['k'] in replace:
            for mod,value in replace[tag.attrib['k']].iteritems():
                tag.attrib['v'] = re.sub(mod,value,tag.attrib['v'])
        
    
    ET.ElementTree(root).write(file_tagok, xml_declaration=True,encoding="UTF-8")

print "Apply correction"
applymod()
print "New file created"


Apply correction
New file created


In [12]:
## contents 8 ## 

# Recheck content

checktagcontent()


Start : 2016-12-08 10:37:53.513000 / Finish : 2016-12-08 10:38:01.807000 / Duration : 0:00:08.294000

{'addr:city / Colomiers': 28,
 'addr:city / Cugnaux': 390,
 'addr:city / La Salvetat Saint Gilles': 1,
 'addr:city / Pibrac': 1,
 'addr:city / Plaisance du Touch': 4,
 'addr:city / Toulouse': 1467,
 'addr:city / Tournefeuille': 53,
 'addr:country / FR': 2281,
 'addr:housename / 30': 1,
 'addr:housename / Airbus Operations SAS': 1,
 'addr:housename / Batiment A': 1,
 'addr:housename / Batiment B': 1,
 'addr:housename / Batiment Kappa': 1,
 "addr:housename / Bocage de l'Armurie Batiment A": 1,
 "addr:housename / Bocage de l'Armurie Batiment B": 1,
 'addr:housename / D22': 1,
 'addr:housename / D26': 1,
 'addr:housename / Europarc B1': 1,
 'addr:housename / Europarc B10': 1,
 'addr:housename / Europarc B2': 1,
 'addr:housename / Europarc B3': 1,
 'addr:housename / Europarc B4': 1,
 'addr:housename / Europarc B6': 1,
 'addr:housename / Europarc B7': 1,
 'addr:housename / Europarc B8': 1,
 

In [13]:
## contents 9 ## 

## Convert to JSON and store output in a file ##

def encodejson(elem, event):
    global result_list
    
    item = {}
    tag = {}
    member = []
    nd = []
    addr = {}
    
    # Insert a doctype attribute to distinguish between node, ways and relations
    item['doctype'] = elem.tag
    
    # Put attributes stored as text
    for key in ['id','version','changeset','visible','uid','user','timestamp']:
        if key in elem.attrib:
            item[key] = elem.attrib[key]
    
    # Put attributes stored float
    for key in ['lon','lat']:
        if key in elem.attrib:
            item[key] = float(elem.attrib[key])
            
    for subelem in elem:
        if subelem.tag == "tag":
            key = subelem.attrib['k']
            # All addr fields will be stored in a sub element
            if key[:5]== "addr:":
                addr[key[5:]] = subelem.attrib['v']
            else:
                tag[subelem.attrib['k']] = subelem.attrib['v']
        elif subelem.tag == "nd":
            # all nd tags will be stored in a dedicated list
            nd.append(subelem.attrib['ref'])
        else:
            # Each member will be stored in a subelement
            onemember = {}
            onemember['type'] = subelem.attrib['type']
            onemember['ref'] = subelem.attrib['ref']
            onemember['role'] = subelem.attrib['role']
            member.append(onemember)            
    
    # Only store sub elements and lists if they contains data
    if len(addr) >0:
        item['addr'] = addr
    if len(tag) > 0:
        item['tag'] = tag
    if len(nd) > 0:
        item['nd'] = nd
    if len(member) > 0:
        item['member'] = member

    result_list.append(item)
    count(elem.tag + ' created',result_dict)
       
context = ET.iterparse(file_tagok, events=('end',),tag=('way','node','relation'))
parse(context,encodejson)

# Save json file
with open(file_json, "w") as outfile:    
    json.dump(result_list, outfile, indent=2)


Start : 2016-12-08 10:38:02.840000 / Finish : 2016-12-08 10:38:25.527000 / Duration : 0:00:22.687000

{'node created': 447694, 'relation created': 465, 'way created': 73067}



In [14]:
## contents 10 ## 

## Store data in mongodb ##

client = MongoClient()
db = client.OpenStreetMap
col = db.tournefeuille

# Delete all collection content and insert all result list content
col.delete_many({})
col.insert_many(result_list)
print "Number of document in mongodb"
print db.tournefeuille.count()

# A faster way to import document is to use the mongoimport utility
# mongoimport" --db OpenStreeMap --collection tournefeuille --drop --jsonArray --file tournefeuille.json



Number of document in mongodb
521226


In [15]:
## contents 11 ##

## Perform mongodb query to check data consistency

print "Number of docs per doc type"
aggr = col.aggregate([
        { "$group": { "_id" : "$doctype", "number" : { "$sum" : 1 } } }        
    ])

pp.pprint(list(aggr))
print

print "Number of nodes having an adress"
aggr = col.aggregate([
        { "$match" : { "doctype" : "node", "addr" : { "$exists": 1 }}},
        { "$group": { "_id" : "$doctype", "number" : { "$sum" : 1 } } }        
    ])

pp.pprint(list(aggr))
print

print "Number of nodes having an adress zip code but no city"
aggr = col.aggregate([
        { "$match" : { "doctype" : "node", "addr.postcode" : { "$exists": 1 }, "addr.city" : { "$exists": 0 }}},
        { "$group": { "_id" : "$addr.postcode", "number" : { "$sum" : 1 } } }        
    ])

pp.pprint(list(aggr))
print



Number of docs per doc type
[{u'_id': u'node', u'number': 447694},
 {u'_id': u'relation', u'number': 465},
 {u'_id': u'way', u'number': 73067}]

Number of nodes having an adress
[{u'_id': u'node', u'number': 4716}]

Number of nodes having an adress zip code but no city
[{u'_id': u'31100', u'number': 1},
 {u'_id': u'31170', u'number': 9},
 {u'_id': u'31830', u'number': 2},
 {u'_id': u'31770', u'number': 8}]



In [16]:
## contents 12 ##

# Add city names when missing

col.update_many({ "addr.postcode" : "31170"} , { "$set" : {'addr.city' : 'Tournefeuille'}})
col.update_many({ "addr.postcode" : "31770"} , { "$set" : {'addr.city' : 'Colomiers'}})
col.update_many({ "addr.postcode" : "31830"} , { "$set" : {'addr.city' : 'Plaisance du Touch'}})
col.update_many({ "addr.postcode" : "31100"} , { "$set" : {'addr.city' : 'Toulouse'}})

print "Number of nodes having an adress zip code but no city (should be normally empty)"
aggr = col.aggregate([
        { "$match" : { "doctype" : "node", "addr.postcode" : { "$exists": 1 }, "addr.city" : { "$exists": 0 }}},
        { "$group": { "_id" : "$addr.postcode", "number" : { "$sum" : 1 } } }        
    ])
pp.pprint(list(aggr))


Number of nodes having an adress zip code but no city (should be normally empty)
[]


In [17]:
## contents 13 ##

# Identify for each nodes the number of linked ways and relations

print "Start update"

ways = col.aggregate([
    { "$match" : { "doctype" : "way"}},
    { "$project" : { "uid" : 1 , "nd" : 1}},
    { "$unwind" : "$nd"},
    { "$group" : { "_id" : "$nd", "count" : { "$sum" : 1}}}
    ])

# We won't do one query per node, but one query per number of associated ways
# So we create a list of nodes per ways numbers
groups = {}
for way in ways:
    key = way['count']
    if key in groups:
        group = groups[key]
    else:
        group = []    
    group.append(way['_id'])
    groups[key] = group

# And we now update all related nodes
for key in groups:
    print "Update ways : " + str(key) + " - Number of affected nodes : " + str(len(groups[key]))
    result = col.update_many({ "doctype" : "node", "id" : { "$in" : groups[key]}} , { "$set" : {'linked_ways' : key}})
    pp.pprint("Number of modified document : " + str(result.modified_count))
    
rels = col.aggregate([
    { "$match" : { "doctype" : "relation", "member" : { "$exists" : 1}}},
    { "$project" : { "uid" : 1 , "member" : 1}},
    { "$unwind" : "$member"},
    { "$group" : { "_id" : "$member.ref", "count" : { "$sum" : 1}}}
    ])

# We won't do one query per node, but one query per number of associated relations
# So we create a list of nodes per relations numbers
groups = {}
for rel in rels:
    key = rel['count']
    if key in groups:
        group = groups[key]
    else:
        group = []    
    group.append(rel['_id'])
    groups[key] = group

# And we now update all related nodes
for key in groups:
    print "Update relations : " + str(key) + " - Number of affected nodes : " + str(len(groups[key]))
    result = col.update_many({ "doctype" : "node", "id" : { "$in" : groups[key]}} , { "$set" : {'linked_relations' : key}})
    pp.pprint("Number of modified document : " + str(result.modified_count))
    

Start update
Update ways : 1 - Number of affected nodes : 299671
'Number of modified document : 299671'
Update ways : 2 - Number of affected nodes : 110334
'Number of modified document : 110334'
Update ways : 3 - Number of affected nodes : 26315
'Number of modified document : 26315'
Update ways : 4 - Number of affected nodes : 4209
'Number of modified document : 4209'
Update ways : 5 - Number of affected nodes : 319
'Number of modified document : 319'
Update ways : 6 - Number of affected nodes : 61
'Number of modified document : 61'
Update ways : 7 - Number of affected nodes : 11
'Number of modified document : 11'
Update relations : 1 - Number of affected nodes : 9598
'Number of modified document : 4686'
Update relations : 2 - Number of affected nodes : 1333
'Number of modified document : 97'
Update relations : 3 - Number of affected nodes : 374
'Number of modified document : 44'
Update relations : 4 - Number of affected nodes : 291
'Number of modified document : 29'
Update relations :

In [18]:
## contents 14 ##

# Get number of nodes not reference at all (no ways, no relations)

nodes = col.aggregate([
    { "$match" : { "doctype" : "node" , "linked_ways" : { "$exists" : 0}, "linked_relations" : { "$exists" : 0}}},
    { "$group" : { "_id" : None, "count" : { "$sum" : 1}}},
    ])

print "Nodes without associateds ways nor relations"
pp.pprint(list(nodes))


Nodes without associateds ways nor relations
[{u'_id': None, u'count': 2342}]
